In [3]:
import numpy as np
import pandas as pd
target_user_id=pd.read_csv('/python/recommender_system/target_user_id.csv')
validation_test_data=pd.read_csv('/python/recommender_system/validation_testing_data.csv')
final_final=pd.read_csv('/python/recommender_system/final_final.csv')
df_news_all=pd.read_csv('/python/recommender_system/df_news_all.csv')

from scipy import spatial
from tqdm import tqdm_notebook

import re
import os
import jieba
import jieba.posseg as pseg
import pickle

from gensim.models import Doc2Vec, doc2vec
from gensim.models import word2vec
import random
import logging

from itertools import zip_longest

FileNotFoundError: [Errno 2] File b'recommender_system/target_user_id.csv' does not exist: b'recommender_system/target_user_id.csv'

In [ ]:
bert_predict_output=pd.read_csv('/project/at081-group38/AT081193_ER/project-BERt/bert/tmp/news_output/test_output/test_results.tsv',sep='\t')
bert_test_data_with_target = pd.read_csv('/project/at081-group38/AT081193_ER/bert_test_data_with_target.csv')
#取得變成columns name的資料
temp=bert_predict_output.columns.values

#轉成dataFrame
d = {'notClick':[temp[0]], 'Click':[temp[1]]} 
first_df=pd.DataFrame(d)

#將原本的欄位重新命名
bert_predict_output.columns=['notClick','Click']

#把兩個資料合併，注意 concat的第一個資料集須放變成欄位名稱的資料，不然位置會錯
bert_predict_output=pd.concat([first_df, bert_predict_output],ignore_index=True)
bert_predict_output.head()

#把整理好的資料再跟bert_test_data水平合併
bert_result = pd.concat([bert_test_data_with_target, bert_predict_output],axis=1)
bert_result = bert_result.drop(['label'],axis=1)

In [ ]:
bert_result[["notClick", "Click"]] = bert_result[["notClick", "Click"]].apply(pd.to_numeric)

In [ ]:
# # 這裡暫不使用
# target_user_id=target_csv.iloc[0,:].values[0]
# pred_time = validation_test_data[validation_test_data.user_id == target_user_id].iloc[0,:].time

# bert_result_date = bert_result.date
# for i in range(len(bert_result)):
#     if (pred_time-bert_result_date[i])>200000000: # 若該新聞發布日期和推薦時間超過兩個月則刪除該候選新聞
#         bert_result = bert_result.drop([i])
# bert_result = bert_result.reset_index(drop=True)

In [ ]:
bert_result = bert_result.sort_values(by="Click" , ascending=False)
bert_result = bert_result.reset_index(drop=True)

In [ ]:
bert_result.head(2)

In [ ]:
len(bert_result)

In [ ]:
#################### 選擇分數最高的前 n 筆
n = 1000
####################
bert_result = bert_result.drop(bert_result.index.values[(bert_result.index.values>n-1).tolist()].tolist())

In [ ]:
target_user_id_str=target_user_id.iloc[0,:].values[0]
validation_data_guid = validation_test_data[validation_test_data.user_id == target_user_id_str].iloc[0,:].guid
if bert_result[bert_result.news_guid==validation_data_guid].empty:
    print('目標不存在候選集合當中')

else:
    print('目標已存在候選集合當中')

## 將最後一筆觀看紀錄加入資料當中

In [ ]:
final_final.head(2)

In [ ]:
final_final = final_final.loc[:,'time':'news_id']

In [ ]:
# 找出該 user 最後一筆看的 news
particular_user_last_news_id = final_final[final_final.user_id == target_user_id_str].tail(1).news_id.values[0]

In [ ]:
df_news_all.head(2)

In [ ]:
particular_user_last_news = df_news_all[df_news_all.news_guid == particular_user_last_news_id]

In [ ]:
particular_user_last_news

In [ ]:
bert_result = bert_result.drop(['notClick', 'Click'], axis = 1)

In [ ]:
bert_result.head(2)

In [ ]:
bert_result_with_user_last_news = pd.concat([particular_user_last_news, bert_result], axis = 0, ignore_index = True)

In [ ]:
bert_result_with_user_last_news.head(2)

# Bert

In [ ]:
title=[]
content=[]
title_content=[]

for index,row in tqdm_notebook(bert_result_with_user_last_news.iterrows()):
    title.append(row['title'])
    content.append(row['content_remove_html_tag'])
    title_content.append(row['title']+' '+row['content_remove_html_tag'])
    

## remove html tag

In [ ]:
# 處理 title 的 html
li = []
for i in range(len(title)):
    li.append(" ".join(title[i].split()))
title = li

In [ ]:
# 處理 content 的 html
li = []
for i in range(len(content)):
    li.append(" ".join(content[i].split()))
content = li

In [ ]:
# 處理 content 的 html
li = []
for i in range(len(title_content)):
    li.append(" ".join(title_content[i].split()))
title_content = li

In [ ]:
sp='\n'
title_s=sp.join(title)
content_s=sp.join(content)
title_content_s=sp.join(title_content)

In [ ]:
with open("/project/at081-group38/AT081193_ER/title_s.txt", "w") as text_file:
    text_file.write(title_s)

In [ ]:
with open("/project/at081-group38/AT081193_ER/content_s.txt", "w") as text_file:
    text_file.write(content_s)

In [ ]:
with open("/project/at081-group38/AT081193_ER/title_content_s.txt", "w") as text_file:
    text_file.write(title_content_s)

#######################################################################################
######################################################################################

layers=-1,-2,-3,-4 \

In [ ]:
!python extract_features.py \
  --input_file=/project/at081-group38/AT081193_ER/title_content_s.txt \
  --output_file=tmp/output.jsonl \
  --vocab_file=chinese_L-12_H-768_A-12/vocab.txt \
  --bert_config_file=chinese_L-12_H-768_A-12/bert_config.json \
  --init_checkpoint=tmp/news_output/model.ckpt-18200.index \
  --layers=-1 \
  --max_seq_length=128 \
  --batch_size=8

In [ ]:
import json

data = []
with open('tmp/output.jsonl') as f:
    for line in f:
        data.append(json.loads(line))

In [ ]:
li = []
for i in range(len(data)):
    result = 1 - spatial.distance.cosine(data[0]['features'][0]['layers'][0]['values'], data[i]['features'][0]['layers'][0]['values'])
    li.append(result)

In [ ]:
cosine_similarity_bert_feature = pd.DataFrame(li)
cosine_similarity_bert_feature.columns = ['cosine_similarity']
cosine_similarity_bert_feature = cosine_similarity_bert_feature.sort_values(by = ['cosine_similarity'], ascending=False)

In [ ]:
final_candidate_bert_feature = bert_result_with_user_last_news.iloc[cosine_similarity_bert_feature.index[1:400],:]

In [ ]:
#如果不存在裡面的話，就把加入Validation_data
validation_data_guid = validation_test_data[validation_test_data.user_id == target_user_id_str].iloc[0,:].guid
if final_candidate_bert_feature[final_candidate_bert_feature.news_guid==validation_data_guid].empty:
    print('目標不存在候選集合當中')
else:
    print('目標已存在候選集合當中')

In [ ]:
# os.chdir("../") # 指定工作路徑
# os.getcwd() # 返回當前工作路徑

In [ ]:
## set dictionary (can define yourself)
jieba.set_dictionary('/project/at081-group38/AT081193_ER/jieba/dict.txt.big')
stop_words = open('/project/at081-group38/AT081193_ER/jieba/stop_words.txt', encoding="utf-8").read().splitlines()

In [ ]:
## filter rules
bert_result_with_user_last_news['content_remove_html_tag'] = bert_result_with_user_last_news['content_remove_html_tag'].str.replace('https?:\/\/\S*', '')
bert_result_with_user_last_news['content_remove_html_tag'] = bert_result_with_user_last_news['content_remove_html_tag'].replace('', np.nan)

In [ ]:
## remove data
bert_result_with_user_last_news = bert_result_with_user_last_news.dropna()
bert_result_with_user_last_news = bert_result_with_user_last_news.reset_index(drop=True)
bert_result_with_user_last_news['idx'] = bert_result_with_user_last_news.index

In [ ]:
bert_result_with_user_last_news.head(2)

In [ ]:
bert_result_with_user_last_news = bert_result_with_user_last_news.drop(['path', 'news_content'], axis = 1)

In [ ]:
bert_result_with_user_last_news['title_content'] = bert_result_with_user_last_news[['content_remove_html_tag', 'title']].apply(lambda x: ''.join(x), axis=1)

In [ ]:
bert_result_with_user_last_news.columns = ['date', 'news_guid', 'title', 'content', 'idx', 'title_content']

In [ ]:
bert_result_with_user_last_news = bert_result_with_user_last_news.reindex(columns=['date', 'news_guid', 'title', 'content',  'title_content', 'idx'])

In [ ]:
bert_result_with_user_last_news.head(2)

## 先進行斷詞 (選擇 title、 content、 title_content)

In [ ]:
# 可選擇 title、 content、 title_content
content_data = bert_result_with_user_last_news['title_content'].tolist()

In [ ]:
sentences = []

for i, text in enumerate(tqdm_notebook(content_data)):
    line = []

    for w in jieba.cut(text, cut_all=False):
        
        ## remove stopwords and digits
        ## can define your own rules
        if w not in stop_words and not bool(re.match('[0-9]+', w)):
            line.append(w)

    sentences.append(line)

In [ ]:
# print(sentences[0:5])

In [ ]:
#######################################################
cut_data = 'content_cutted'
########################################################
## save data as pickle format
with open('/project/at081-group38/AT081193_ER/'+ cut_data, "wb") as file:
    pickle.dump(sentences, file)

In [ ]:
# 通過 logging.basicConfig 函數對日誌的輸出格式及方式做相關的配置
logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s')
logging.root.setLevel(level=logging.INFO)

In [ ]:
## load 'content_cutted'
with open('/project/at081-group38/AT081193_ER/'+ cut_data, 'rb') as file:
    data = pickle.load(file)

## average word vec

In [ ]:
# build word2vec
# sg=0 CBOW ; sg=1 skip-gram
model = word2vec.Word2Vec(size=256, min_count=5, window=5, sg=0)

In [ ]:
# build vocabulary
model.build_vocab(data)

In [ ]:
# train word2vec model ; shuffle data every epoch
for i in range(20):
    random.shuffle(data)
    model.train(data, total_examples=len(data), epochs=1)

In [ ]:
# ## save model
# model.save('/project/at081-group38/AT081193_ER/word2vec_model/CBOW')

In [ ]:
# ## load word2vec model
# model = word2vec.Word2Vec.load('/project/at081-group38/AT081193_ER/word2vec_model/CBOW')

In [ ]:
## filter words that not in word2vec's vocab
data_filtered = [[w for w in l if w in model.wv] for l in data]

In [ ]:
## compute average word vector
avg_vector = []

for l in data_filtered:
    if len(l)==0:
        avg_vector.append(np.array([0]*256))
    else:
        avg_vector.append(np.mean([model.wv[w] for w in l], axis=0))

In [ ]:
# ## print result
# avg_vector[0]

In [ ]:
# ## save result
# with open('/project/at081-group38/AT081193_ER/word2vec_model/'+cut_data+'avg_vector', 'wb') as file:
#     pickle.dump(avg_vector, file)

In [ ]:
# ## load average word2vec result
# with open('/project/at081-group38/AT081193_ER/word2vec_model/'+cut_data+'avg_vector', 'rb') as file:
#     avg_vector = pickle.load(file)

In [ ]:
li = []
for i in range(len(avg_vector)):
    result = 1 - spatial.distance.cosine(avg_vector[0], avg_vector[i])
    li.append(result)

In [ ]:
cosine_similarity_average_word_vec = pd.DataFrame(li)
cosine_similarity_average_word_vec.columns = ['cosine_similarity']
cosine_similarity_average_word_vec = cosine_similarity_average_word_vec.sort_values(by = ['cosine_similarity'], ascending=False)

In [ ]:
final_candidate_average_word_vec = bert_result_with_user_last_news.iloc[cosine_similarity_average_word_vec.index[1:400],:]

In [ ]:
validation_data_guid = validation_test_data[validation_test_data.user_id == target_user_id_str].iloc[0,:].guid
if final_candidate_average_word_vec[final_candidate_average_word_vec.news_guid==validation_data_guid].empty:
    print('目標不存在候選集合當中')
else:
    print('目標已存在候選集合當中')

# doc2vec

In [ ]:
## create a document id map
sentence_list = []

for i, l in enumerate(data):
    sentence_list.append(doc2vec.LabeledSentence(words=l, tags=[str(i)]))

In [ ]:
# ## print result
# sentence_list[0]

In [ ]:
## define 轉換器
model = Doc2Vec(size=256, min_count=5, window=4)

In [ ]:
## build vocabulary
model.build_vocab(sentence_list)

In [ ]:
# train word2vec model ; shuffle data every epoch
for i in range(20):
    random.shuffle(sentence_list)
    model.train(sentence_list, total_examples=len(data), epochs=1)

In [ ]:
# ## print result
# model.docvecs[0]

In [ ]:
# ## save model
# model.save('/project/at081-group38/AT081193_ER/word2vec_model/doc2vec')

In [ ]:
# ## load word2vec model
# model = word2vec.Word2Vec.load('/project/at081-group38/AT081193_ER/word2vec_model/doc2vec')

In [ ]:
li = []
for i in range(len(model.docvecs)):
    result = 1 - spatial.distance.cosine(model.docvecs[0], model.docvecs[i])
    li.append(result)

In [ ]:
cosine_similarity_doc2vec = pd.DataFrame(li)
cosine_similarity_doc2vec.columns = ['cosine_similarity']
cosine_similarity_doc2vec = cosine_similarity_doc2vec.sort_values(by = ['cosine_similarity'], ascending=False)

In [ ]:
final_candidate_doc2vec = bert_result_with_user_last_news.iloc[cosine_similarity_doc2vec.index[1:400],:]

In [ ]:
validation_data_guid = validation_test_data[validation_test_data.user_id == target_user_id_str].iloc[0,:].guid
if final_candidate_doc2vec[final_candidate_doc2vec.news_guid==validation_data_guid].empty:
    print('目標不存在候選集合當中')
else:
    print('目標已存在候選集合當中')

## bag of words

In [ ]:
content_data = pd.DataFrame(content_data, columns=['content'])

In [ ]:
## word count
## http://blog.csdn.net/gatieme/article/details/43235791 (中文正則表達式)
content_data['word_count'] = content_data['content'].str.count('[a-zA-Z0-9]+') + content_data['content'].str.count('[\u4e00-\u9fff]')

In [ ]:
## punctuation count
content_data['punctuation'] = content_data['content'].str.replace('[\w\s]', '')
content_data['punctuation_count'] = content_data['punctuation'].str.len()

In [ ]:
## question mark count
content_data['question_count'] = content_data['punctuation'].str.count('[?？]')

In [ ]:
## drop punctuation column
content_data = content_data.drop(['punctuation'],axis=1)

In [ ]:
# # punctuation 是標點符號
# content_data.head()

In [ ]:
# ## compute correlation
# content_data.iloc[:, 1:].corr()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
## define transformer (轉換器)
vectorizer = CountVectorizer()
count = vectorizer.fit_transform([' '.join(x) for x in data])

In [ ]:
## save data as pickle format
with open('/project/at081-group38/AT081193_ER/word2vec_model/news_count', "wb") as file:
    pickle.dump([vectorizer, count], file)

In [ ]:
## load bag of words result
with open('/project/at081-group38/AT081193_ER/word2vec_model/news_count', 'rb') as file:
    _, count = pickle.load(file)

In [ ]:
## select top 256 words (counts of document) 
most_count_id = np.array((count > 0).sum(axis=0))[0].argsort()[::-1][:256]

In [ ]:
## subset data
count = count[:, most_count_id]

In [ ]:
count_data = np.zeros((content_data.shape[0], 256))

In [ ]:
## subset bag of words matrix
count_data = count[count_data[:, 0]].toarray()

In [ ]:
li = []
for i in range(len(count_data)):
    result = 1 - spatial.distance.cosine(count_data[0], count_data[i])
    li.append(result)

In [ ]:
cosine_similarity_bag_of_words = pd.DataFrame(li)
cosine_similarity_bag_of_words.columns = ['cosine_similarity']
cosine_similarity_bag_of_words = cosine_similarity_bag_of_words.sort_values(by = ['cosine_similarity'], ascending=False)

In [ ]:
final_candidate_bag_of_words = bert_result_with_user_last_news.iloc[cosine_similarity_bag_of_words.index[1:400],:]

In [ ]:
validation_data_guid = validation_test_data[validation_test_data.user_id == target_user_id_str].iloc[0,:].guid
if final_candidate_bag_of_words[final_candidate_bag_of_words.news_guid==validation_data_guid].empty:
    print('目標不存在候選集合當中')
else:
    print('目標已存在候選集合當中')

## TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
## define transformer (轉換器)
vectorizer = TfidfVectorizer(norm=None) ## do not do normalize
tfidf = vectorizer.fit_transform([' '.join(x) for x in data])

In [ ]:
## save data as pickle format
with open('/project/at081-group38/AT081193_ER/word2vec_model/news_tfidf', "wb") as file:
    pickle.dump([vectorizer, tfidf], file)

### select top 10 average tf-idf of words

In [ ]:
# create a dictionary: id as key ; word as values
# id2word = {v:k for k, v in vectorizer.vocabulary_.items()}

In [ ]:
# ## columnwise average: words tf-idf
# avg = tfidf.sum(axis=0) / (tfidf!=0).sum(axis=0)

# ## set df < 20 as 0
# avg[(tfidf!=0).sum(axis=0)<20] = 0

In [ ]:
# avg = np.array(avg)[0]

In [ ]:
# ## top 10 tfidf's wordID
# most_avg_id = avg.argsort()[::-1][:10].tolist()
# most_avg_id

In [ ]:
# ## print top 10 tf-idf's words
# features = [id2word[i] for i in most_avg_id]
# features

In [ ]:
## load tf-idf result
with open('/project/at081-group38/AT081193_ER/word2vec_model/news_tfidf', 'rb') as file:
    _, tfidf = pickle.load(file)

In [ ]:
## select top 256 words (counts of document) 
most_tfidf_id = np.array((tfidf > 0).sum(axis=0))[0].argsort()[::-1][:256]

In [ ]:
## subset data
tfidf = tfidf[:, most_tfidf_id]

In [ ]:
tfidf_data = np.zeros((content_data.shape[0], 256))

In [ ]:
## subset tf-idf matrix
tfidf_data = tfidf[tfidf_data[:, 0]].toarray()

In [ ]:
li = []
for i in range(len(tfidf_data)):
    result = 1 - spatial.distance.cosine(count_data[0], count_data[i])
    li.append(result)

In [ ]:
cosine_similarity_TF_IDF = pd.DataFrame(li)
cosine_similarity_TF_IDF.columns = ['cosine_similarity']
cosine_similarity_TF_IDF = cosine_similarity_TF_IDF.sort_values(by = ['cosine_similarity'], ascending=False)

In [ ]:
final_candidate_TF_IDF = bert_result_with_user_last_news.iloc[cosine_similarity_TF_IDF.index[1:400],:]

In [ ]:
validation_data_guid = validation_test_data[validation_test_data.user_id == target_user_id_str].iloc[0,:].guid
if final_candidate_TF_IDF[final_candidate_TF_IDF.news_guid==validation_data_guid].empty:
    print('目標不存在候選集合當中')
else:
    print('目標已存在候選集合當中')

# Hit rate

In [ ]:
# 總篩選篇數約48000筆
total_news_num = 48000
# 後選集合佔總篇數的百分比
percentage = [0.25/100, 0.5/100, 1/100, 1.5/100, 2/100]
# 候選集合數 [120, 240, 480, 720, 960]
n = ( np.array(percentage)*total_news_num ).tolist()
score = [1, 0.9, 0.8, 0.7, 0.6]

In [ ]:
li = []
for i in n:
    final_candidate_bert_feature = bert_result_with_user_last_news.iloc[cosine_similarity_bert_feature.index[1:int(i)],:]

    validation_data_guid = validation_test_data[validation_test_data.user_id == target_user_id_str].iloc[0,:].guid
    if final_candidate_bert_feature[final_candidate_bert_feature.news_guid==validation_data_guid].empty:
        li.append(0)
    else:
        li.append(1)
        break
    
hit_rate_bert = sum([x*y for x,y in zip_longest(score, li, fillvalue=0)])    
hit_rate_bert

In [ ]:
li = []
for i in n:
    final_candidate_average_word_vec = bert_result_with_user_last_news.iloc[cosine_similarity_average_word_vec.index[1:int(i)],:]
    
    validation_data_guid = validation_test_data[validation_test_data.user_id == target_user_id_str].iloc[0,:].guid
    if final_candidate_average_word_vec[final_candidate_average_word_vec.news_guid==validation_data_guid].empty:
        li.append(0)
    else:
        li.append(1)
        break
hit_rate_average_word_vec= sum([x*y for x,y in zip_longest(score, li, fillvalue=0)])    
hit_rate_average_word_vec

In [ ]:
li = []
for i in n:
    final_candidate_doc2vec = bert_result_with_user_last_news.iloc[cosine_similarity_doc2vec.index[1:int(i)],:]
    
    validation_data_guid = validation_test_data[validation_test_data.user_id == target_user_id_str].iloc[0,:].guid
    if final_candidate_doc2vec[final_candidate_doc2vec.news_guid==validation_data_guid].empty:
        li.append(0)
    else:
        li.append(1)
        break
hit_rate_doc2vec= sum([x*y for x,y in zip_longest(score, li, fillvalue=0)])    
hit_rate_doc2vec   

In [ ]:
li = []
for i in n:
    final_candidate_bag_of_words = bert_result_with_user_last_news.iloc[cosine_similarity_bag_of_words.index[1:int(i)],:]
    
    validation_data_guid = validation_test_data[validation_test_data.user_id == target_user_id_str].iloc[0,:].guid
    if final_candidate_bag_of_words[final_candidate_bag_of_words.news_guid==validation_data_guid].empty:
        li.append(0)
    else:
        li.append(1)
        break
hit_rate_bag_of_words = sum([x*y for x,y in zip_longest(score, li, fillvalue=0)])    
hit_rate_bag_of_words         

In [ ]:
li = []
for i in n:
    final_candidate_TF_IDF = bert_result_with_user_last_news.iloc[cosine_similarity_TF_IDF.index[1:int(i)],:]

    validation_data_guid = validation_test_data[validation_test_data.user_id == target_user_id_str].iloc[0,:].guid
    if final_candidate_TF_IDF[final_candidate_TF_IDF.news_guid==validation_data_guid].empty:
        li.append(0)
    else:
        li.append(1)
        break
hit_rate_TF_IDF = sum([x*y for x,y in zip_longest(score, li, fillvalue=0)])    
hit_rate_TF_IDF           